In [1]:
import re
import nltk
from urllib.parse import unquote
# 缩小向量空间
def GeneSeg(payload):
    # lowercase
    payload=payload.lower()
    payload=unquote(payload)
    payload,num=re.subn(r'\d+',"0",payload)
    payload,num=re.subn(r'(http|https)://[a-zA-Z0-9\.@&/#!#\?]+',"http://u", payload)
    payload,num=re.subn(r'\/\*.?\*\/',"",payload)
    r = '''
        (?x)[\w\.]+?\(
        |\)
        |"\w+?"
        |'\w+?'
        |http://\w
        |</\w+>
        |<\w+>
        |<\w+
        |\w+=
        |>
       |[\w\.]+
    '''
    return nltk.regexp_tokenize(payload,r)

import csv
csv_reader = csv.reader(open("./dmzo_nomal.csv",'r'))
content = []
target=[]
for line in csv_reader:
    target.append(1)
    content.append(line)
csv_reader = csv.reader(open("./xssed.csv",'r'))
for line in csv_reader:
    target.append(0)
    content.append(line)

In [2]:
result_word = []
for i in content:
    a = GeneSeg(i[0])
    result_word.append(a)
del(a,content,i,line)

In [3]:
result_word[2]

['mode=', 'navigation', 'amp', 'categoryid=', '0']

In [4]:
sentences = []
for i in result_word:
    sentence = ""
    for j in i:
        sentence += j
        sentence += " "
    sentences.append(sentence)
del(result_word)
del(sentence,i,j)

In [5]:
sentences[0]

'sid= amp ring= hentff0 amp id= amp list '

In [6]:
from keras.preprocessing.text import Tokenizer
# 在30个单词后截断指令
max_lens = 30
# 只考虑最常用的10000
word_max = 10000
tokenizer = Tokenizer(num_words=word_max)
tokenizer.x(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [7]:
sentences[0]

'sid= amp ring= hentff0 amp id= amp list '

In [8]:
train_samples = int(0.75*len(sentences))
test_samples = len(sentences) - train_samples
import numpy as np
from keras.preprocessing.sequence import pad_sequences
word_index = tokenizer.word_index
data = pad_sequences(sequences,maxlen=max_lens)
labels = np.asarray(target)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

X_train = data[:train_samples]
y_train = labels[:train_samples]
X_test = data[train_samples:]
y_test = labels[train_samples:]

In [9]:
from keras.models import Sequential
from keras.layers import Embedding,Dense
from keras.layers import LSTM
model = Sequential()
model.add(Embedding(word_max,8,input_length=max_lens))
model.add(LSTM(32))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.summary()
history = model.fit(X_train,y_train,epochs=10,batch_size=32,validation_split=0.2)


W1015 19:20:27.152863 4586128832 deprecation_wrapper.py:119] From /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1015 19:20:27.198287 4586128832 deprecation_wrapper.py:119] From /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1015 19:20:27.211700 4586128832 deprecation_wrapper.py:119] From /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1015 19:20:27.427363 4586128832 deprecation_wrapper.py:119] From /anaconda3/lib/python3.6/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1015 19:20:27.441509 4586128832 deprecation_wrapper.py:119] From /anaconda3

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 8)             80000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                5248      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 85,281
Trainable params: 85,281
Non-trainable params: 0
_________________________________________________________________


W1015 19:20:28.138639 4586128832 deprecation_wrapper.py:119] From /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 38899 samples, validate on 9725 samples
Epoch 1/10
38899/38899 [==============================] - 27s 691us/step - loss: 0.0866 - acc: 0.9745 - val_loss: 0.0463 - val_acc: 0.9877
Epoch 2/10
38899/38899 [==============================] - 25s 636us/step - loss: 0.0378 - acc: 0.9899 - val_loss: 0.0415 - val_acc: 0.9891
Epoch 3/10
38899/38899 [==============================] - 24s 612us/step - loss: 0.0365 - acc: 0.9905 - val_loss: 0.0391 - val_acc: 0.9900
Epoch 4/10
38899/38899 [==============================] - 24s 617us/step - loss: 0.0361 - acc: 0.9904 - val_loss: 0.0406 - val_acc: 0.9902
Epoch 5/10
38899/38899 [==============================] - 25s 636us/step - loss: 0.0355 - acc: 0.9906 - val_loss: 0.0380 - val_acc: 0.9903
Epoch 6/10
38899/38899 [==============================] - 26s 662us/step - loss: 0.0345 - acc: 0.9909 - val_loss: 0.0378 - val_acc: 0.9903
Epoch 7/10
38899/38899 [==============================] - 24s 627us/step - loss: 0.0340 - acc: 0.9910 - val_loss: 0.0

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [11]:
rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)

rfc_predict=rfc.predict(X_test)
print(rfc.score(X_test,y_test))

print(classification_report(y_test, rfc_predict, target_names=["died", "survived"]))

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9838361404158183
              precision    recall  f1-score   support

        died       0.98      0.99      0.98      8337
    survived       0.99      0.98      0.98      7872

    accuracy                           0.98     16209
   macro avg       0.98      0.98      0.98     16209
weighted avg       0.98      0.98      0.98     16209

